In [82]:
import numpy as np
import nltk
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [222]:
df = pd.read_csv("spam_ham_dataset.csv")

In [223]:
print(df["label"].value_counts())

ham     3672
spam    1499
Name: label, dtype: int64


In [224]:
df['processed_text']=df['text']

In [225]:
df.head(10)

,Unnamed: 0,label,text,label_num,processed_text
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0,Subject: enron methanol ; meter # : 988291\r\n...
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0,"Subject: hpl nom for january 9 , 2001\r\n( see..."
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0,"Subject: neon retreat\r\nho ho ho , we ' re ar..."
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1,"Subject: photoshop , windows , office . cheap ..."
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0,Subject: re : indian springs\r\nthis deal is t...
5,2949,ham,Subject: ehronline web address change\r\nthis ...,0,Subject: ehronline web address change\r\nthis ...
6,2793,ham,Subject: spring savings certificate - take 30 ...,0,Subject: spring savings certificate - take 30 ...
7,4185,spam,Subject: looking for medication ? we ` re the ...,1,Subject: looking for medication ? we ` re the ...
8,2641,ham,Subject: noms / actual flow for 2 / 26\r\nwe a...,0,Subject: noms / actual flow for 2 / 26\r\nwe a...
9,1870,ham,"Subject: nominations for oct . 21 - 23 , 2000\...",0,"Subject: nominations for oct . 21 - 23 , 2000\..."


In [226]:
stop_words = set(stopwords.words('english'))

In [227]:
def remove_stopwords(df):
    for i in range(len(df)):
        df["processed_text"][i] = df["processed_text"][i].lower().split()
        filtered_sentence = [w for w in df["processed_text"][i] if not w.lower() in stop_words]
        df["processed_text"][i]=' '.join(filtered_sentence)
    return df

In [228]:
def preprocess(df):
    df['processed_text']=df['processed_text'].str[9:]
    df=remove_stopwords(df)
    df['processed_text']=df['processed_text'].str.replace('[^a-z]+',' ').replace('  ',' ')

In [229]:
preprocess(df)
# df.head(10)

<ipython-input-227-6d00cd145a3f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["processed_text"][i] = df["processed_text"][i].lower().split()
<ipython-input-227-6d00cd145a3f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["processed_text"][i]=' '.join(filtered_sentence)
<ipython-input-228-b45d25d1d12c>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['processed_text']=df['processed_text'].str.replace('[^a-z]+',' ').replace('  ',' ')


In [230]:
print(df['processed_text'][0],'\n')
print(df['text'][0],'\n')

enron methanol meter follow note gave monday preliminary flow data provided daren please override pop daily volume presently zero reflect daily activity obtain gas control change needed asap economics purposes  

Subject: enron methanol ; meter # : 988291
this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary
flow data provided by daren } .
please override pop ' s daily volume { presently zero } to reflect daily
activity you can obtain from gas control .
this change is needed asap for economics purposes . 



In [231]:
train_data = df.sample(frac=0.8,random_state=1).reset_index(drop=True)
test_data = df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

print(df['label'].value_counts() / df.shape[0] * 100)

print(train_data['label'].value_counts() / train_data.shape[0] * 100)

print(test_data['label'].value_counts() / test_data.shape[0] * 100)

ham     71.01141
spam    28.98859
Name: label, dtype: float64
ham     70.72758
spam    29.27242
Name: label, dtype: float64
ham     69.052224
spam    30.947776
Name: label, dtype: float64


In [232]:
alpha=1
k=2

In [233]:
p_spam = train_data['label'].value_counts()['spam']/len(train_data)
p_spam

0.2927241962774958

In [234]:
p_ham = train_data['label'].value_counts()['ham']/len(train_data)
p_ham

0.7072758037225042

In [235]:
n_spam = train_data['label'].value_counts()['spam']
n_spam

1211

In [236]:
n_ham = train_data['label'].value_counts()['ham']
n_ham

2926

In [237]:
vocabulary = list(set(train_data['processed_text'].str.split().sum()))

In [ ]:
word_counts_per_sms = pd.DataFrame([
    [row[4].split().count(word) for word in vocabulary]
    for _, row in train_data.iterrows()], columns=vocabulary)
word_counts_per_sms